In [1]:
# Title: Blast Wave simulation
# Author: Eric Henschel
# R = S(gamma) * E^(1/5) * (rho_o)^(-1/5)*t^(2/5)
# initial conditions::
# u(0, r) = 0 m/s
# p(0, r) = 101 kPa
# rho(0, r) = 1.225 kg/m^3


# E = 5.36*rho_o*A^2

In [140]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [79]:
# specific heat of dry air
gamma = 1.4

# initial vel
u_o = 0 # m/s

# initial pressure
p_o = 101000. # kPa

# air density
rho_o = 1.225 # kg/m^3

In [37]:
# list of nuclear weapon energy outputs in kilo-tons of TNT equivalence
# sourced: https://en.wikipedia.org/wiki/Nuclear_weapon_yield

atoms = [{'name': 'Davy Crockett', 'out': 0.02},
        {'name': 'AIR-2 Genie', 'out': 1.5},
        {'name': 'Little Boy', 'out': 18.0},
        {'name': 'Fat Man', 'out': 22.0},
        {'name': 'W76 warhead', 'out': 100.0},
        {'name': 'W87 warhead', 'out': 300.0},
        {'name': 'W88 warhead', 'out': 475.0},
        {'name': 'Ivy King', 'out': 500.0},
        {'name': 'Orange Herald Small', 'out': 720.0},
        {'name': 'B83 nuke', 'out': 1200.0},
        {'name': 'B53 nuke', 'out': 9000.0},
        {'name': 'Tsar Bomba', 'out': 50000.0},
        {'name': 'Sum_Tests', 'out': 510300.0}]

df = pd.DataFrame(atoms)
df

,name,out
0,Davy Crockett,0.02
1,AIR-2 Genie,1.50
2,Little Boy,18.00
3,Fat Man,22.00
4,W76 warhead,100.00
5,W87 warhead,300.00
6,W88 warhead,475.00
7,Ivy King,500.00
8,Orange Herald Small,720.00
9,B83 nuke,1200.00


In [42]:
# Energy output to list; names to list
names = []
energOut = []
for i, row in df.iterrows():
    out = row['out']
    name = row['name']
    energOut.append(out)
    names.append(name)

In [5]:
# Energy Kilotons TNT --> Joules
def energyConversion(E):
    return E * (4.184*10**12)

In [6]:
# radius of shock wave
def radius(t, energy):
    rho_o = 1.225 # kg/m^3
    return (1.031) * energy**(1./5) * rho_o**(-1./5) * t**(2./5)

In [7]:
# B=0.926 * (5/2) is taken from pg. 167 of
# https://www3.nd.edu/~powers/ame.90931/taylor.blast.wave.I.pdf

def velocity(radius, energy):
    rho_o = 1.225 # kg/m^3
    B = 2.31 
    return radius**(-3./2) * energy**(1/2) * (B*rho_o)**(-1./2)

In [8]:
# max pressure
def pressure(radius, energy):
    return 0.155*radius**(-3.)*energy

In [157]:
# simulating 
# radial expansion w.r.t. time
# velocity w.r.t. radius
# pressure w.r.t. radius
simulation = pd.DataFrame()
simsDf = pd.DataFrame()
radsDf = pd.DataFrame()
velsDf = pd.DataFrame()
presDf = pd.DataFrame()

for e, name in zip(energOut, names):
    # data for each Nuke
    E = energyConversion(e)
    rad = []
    vel = [u_o]
    pres = [p_o]
    time = []
    
    # Event loops
    for i in range(100):
        t = float(i)/(10**7)

        r = radius(t, E)
        rad.append(r)
        time.append(t)
        
    for r in rad[1:]:
        v = velocity(r, E)
        vel.append(v)

        p = pressure(r, E)
        pres.append(p)

    # DataFrame construction
    nameArr = np.repeat(name, len(time))
    
    df1 = pd.DataFrame()
    df2 = pd.DataFrame()
    df3 = pd.DataFrame()
    
    # dataframe for radius
    df1['name'] = nameArr
    df1['time'] = time
    df1['radius'] = rad
    pd.concat([radsDf, df1])
    
    # dataframe for velocity
    df2['name'] = nameArr
    df2['radius'] = rad
    df2['velocity'] = vel
    pd.concat(velsDf, df2)

    # dataframe for pressure
    df3['name'] = nameArr
    df3['radius'] = rad
    df3['pressure'] = pres
    pd.concat(presDf, df3)
    

TypeError: first argument must be an iterable of pandas objects, you passed an object of type "DataFrame"

In [155]:
display(velsDf)

""


In [139]:
#simulation

In [116]:
# t = simulation['Sims'][0]['Time']
# r = simulation['Sims'][0]['Radius']
import plotly.express as px
df = px.data.gapminder().query("continent != 'Asia'")
display(df)

# fig = px.line(simulation)
# fig.show()

,country,continent,year,lifeExp,pop,gdpPercap,iso_alpha,iso_num
12,Albania,Europe,1952,55.230,1282697,1601.056136,ALB,8
13,Albania,Europe,1957,59.280,1476505,1942.284244,ALB,8
14,Albania,Europe,1962,64.820,1728137,2312.888958,ALB,8
15,Albania,Europe,1967,66.220,1984060,2760.196931,ALB,8
16,Albania,Europe,1972,67.690,2263554,3313.422188,ALB,8
...,...,...,...,...,...,...,...,...
1699,Zimbabwe,Africa,1987,62.351,9216418,706.157306,ZWE,716
1700,Zimbabwe,Africa,1992,60.377,10704340,693.420786,ZWE,716
1701,Zimbabwe,Africa,1997,46.809,11404948,792.449960,ZWE,716
1702,Zimbabwe,Africa,2002,39.989,11926563,672.038623,ZWE,716
